## Traditional LLL

In [ ]:
#main
from fractions import Fraction
import numpy as np
import timeit


loop = 20


class Vector(list):
    def __init__(self, x):
        super().__init__(x)

    def dot(self, other):
        return sum(map(lambda x: x[0] * x[1], zip(self, Vector(other))))

    def sdot(self):
        return self.dot(self)

    def proj_coff(self, other):
        return self.dot(Vector(other)) / self.sdot()

    def proj(self, other):
        return self.proj_coff(Vector(other)) * self

    def __sub__(self, other):
        return Vector(x - y for x, y in zip(self, Vector(other)))
    
    def __add__(self, other):
        return Vector(x + y for x, y in zip(self, Vector(other)))

    def __mul__(self, other):
        return Vector(x * other for x in self)

    def __rmul__(self, other):
        return Vector(x * other for x in self)



def gramschmidt(v): # returns gram-schmidt orthogonalized vectors
    u = []
    for vi in v:
        ui = Vector(vi)
        for uj in u:
            ui = ui - uj.proj(vi)
        if any(ui): #if any element is non-zero 
            u.append(ui)
    return u
    

def reduction(basis, delta = 0.75) : #our main func
    n = len(basis)
    basis = list(map(Vector, basis))
    ortho = gramschmidt(basis)

    def mu(i, j):   #returns gram-schmidt coeffs
        return ortho[j].proj_coff(basis[i])

    k = 1
    while k < n:
        for j in range(k - 1, -1, -1): #check for all j< k
            mu_kj = mu(k, j)
            if abs(mu_kj) > 0.5: # size condition
                basis[k] = basis[k] - basis[j] * round(mu_kj)
                ortho = gramschmidt(basis)

        if ortho[k].sdot() >= (delta - mu(k, k - 1)**2) * ortho[k - 1].sdot(): #lovasz condition
            k += 1
        else:
            basis[k], basis[k - 1] = basis[k - 1], basis[k] #swap both
            ortho = gramschmidt(basis)
            k = max(k - 1, 1)

    return [list(map(int, b)) for b in basis]

## Improved LLL

In [ ]:
#improved
def gramschmidt_up(ortho,k,basis):
    for i1 in range(k-1,k+1):
        ui = Vector(basis[i1])
        for i2 in range(0,i1):
            ui = ui - ortho[i2].proj(basis[i1])
        if any(ui): #if any element is non-zero 
            ortho[i1] = ui
    return ortho


def reduction_(basis, delta = 0.75) : #our main func
    n = len(basis)
    basis = list(map(Vector, basis))
    ortho = gramschmidt(basis)

    def mu(i, j):   #returns gram-schmidt coeffs
        return ortho[j].proj_coff(basis[i])

    k = 1
    while k < n:
        for j in range(k - 1, -1, -1): #check for all j< k
            mu_kj = mu(k, j)
            if abs(mu_kj) > 0.5: # size condition
                basis[k] = basis[k] - basis[j] * round(mu_kj)

        if ortho[k].sdot() >= (delta - mu(k, k - 1)**2) * ortho[k - 1].sdot(): #lovasz condition
            k += 1
        else:
            basis[k], basis[k - 1] = basis[k - 1], basis[k] #swap both
            ortho = gramschmidt_up(ortho,k,basis)
            k = max(k - 1, 1)

    return [list(map(int, b)) for b in basis]     

## Examples:

In [ ]:
reduction([[47,-215], [95, 460]], 0.75)

[[47, -215], [189, 30]]

In [ ]:
reduction_([[47,215], [95, 460]], 0.75)

[[1, 30], [40, 5]]

In [ ]:
result = timeit.timeit('reduction([[47,215], [95, 460]], 0.75)', globals=globals(), number=loop)
print(result / loop)

0.0011039768199998434


In [ ]:
reduction([[1, 0,0, 1000000], [0, 1, 0, 1414213],[0,0,1,1999998]], 0.75)

[[-2, 0, 1, -2], [-167, -408, 372, 352], [-854, 577, 19, 863]]

In [ ]:
reduction([[6,5],[12,18]])

[[-6, 3], [6, 5]]

In [ ]:
result = timeit.timeit('reduction([[6,5],[12,18]],0.75)', globals=globals(), number=loop)
print(result / loop)

0.0006132918999999788


In [ ]:
#random matrix
A = np.random.randint(-500,500, size=(10,10));A

array([[ 109,  341, -322,   79,  -77,  468, -487, -124,  -57, -151],
       [-146, -133,  440,  117, -106, -224,  490,  306, -415,  394],
       [-408,   33,  481, -208,   62, -161,  -64, -468,  188,  174],
       [-441, -453, -488, -166,  409,   -3, -478,  442,   -1, -215],
       [-224,  255,  100,  -74,  464,  280,  -37,  419, -102,    9],
       [ 139,  333,  -65,  375, -258,  270,  172,   53,   49, -207],
       [ 103, -331,   99, -312,  -73, -463, -117,  -19,  276,  259],
       [ 143, -438, -303, -350, -283,  364,  167, -110, -215,  429],
       [ 459, -233, -173, -401, -208, -220, -442,  126,  358,  342],
       [ 398,  -50, -168,   21,  314, -368,  165,   -5, -455,  245]])

### Below Code segment shows the runtime improvements to traditional LLL algorithm by also proving outputs remain the same.

In [ ]:
print(timeit.timeit('reduction_(list(A))', globals=globals(), number=1)*100)

4.0514125999834505


In [ ]:
print(timeit.timeit('reduction(list(A))', globals=globals(), number=1)*100)

21.283085300092353


In [ ]:
np.array_equal(np.array(reduction(list(A)))-np.array(reduction_(list(A))),np.zeros((10,10)))

True

In [ ]:
reduction(list(A))

[[-59, 292, -151, -30, 105, -147, -71, 178, -52, -22],
 [22, 84, -46, -68, 138, -135, -260, 21, 239, 281],
 [225, -120, 56, 106, 48, 250, 90, 131, 154, 278],
 [-148, 85, 229, -87, -20, -16, -1, 144, 121, -195],
 [242, 2, 34, 63, -331, -193, 55, 34, 325, 52],
 [-247, 243, -50, 168, 58, 225, -162, -269, -139, -234],
 [92, 219, 160, -153, -58, 320, -110, -35, 133, 232],
 [-196, -134, 292, 224, -112, -80, -77, 204, -193, 119],
 [175, -328, -52, 7, -202, 54, -145, 146, 122, -322],
 [-304, 12, -196, -195, 84, 205, -128, 274, 96, 110]]

In [ ]:
reduction_(list(A))

[[-59, 292, -151, -30, 105, -147, -71, 178, -52, -22],
 [22, 84, -46, -68, 138, -135, -260, 21, 239, 281],
 [225, -120, 56, 106, 48, 250, 90, 131, 154, 278],
 [-148, 85, 229, -87, -20, -16, -1, 144, 121, -195],
 [242, 2, 34, 63, -331, -193, 55, 34, 325, 52],
 [-247, 243, -50, 168, 58, 225, -162, -269, -139, -234],
 [92, 219, 160, -153, -58, 320, -110, -35, 133, 232],
 [-196, -134, 292, 224, -112, -80, -77, 204, -193, 119],
 [175, -328, -52, 7, -202, 54, -145, 146, 122, -322],
 [-304, 12, -196, -195, 84, 205, -128, 274, 96, 110]]

## Runtime Analysis

In [ ]:
for size in range(2, 50, 2):
  print(size,timeit.timeit('reduction_(list(np.random.randint(-500,500, size=(size,size))))', globals=globals(), number=100))

2 0.021317060999990645
4 0.1219896659999904
6 0.41546354100000826
8 0.967061770999976
10 1.7714739649999274
12 3.022991384999955
14 4.636518113999955
16 6.566219217000025
18 9.01846535900006
20 11.74763413200003
22 14.577808471000026
24 17.588624621000008
26 21.84994601699998
28 25.394578732000014
30 29.92287170299994
32 32.738089220000006
34 43.10965722100002
36 46.28200038
38 55.28712121900003
40 59.89792254400004
42 67.81528400799994
44 75.62608244900014
46 81.90557611400004
48 91.168513394


In [ ]:
print(size,timeit.timeit('reduction_(list(np.random.randint(-500,500, size=(50,50))))', globals=globals(), number=100))
print(size,timeit.timeit('reduction_(list(np.random.randint(-500,500, size=(55,55))))', globals=globals(), number=100))
print(size,timeit.timeit('reduction_(list(np.random.randint(-500,500, size=(60,60))))', globals=globals(), number=100))
print(size,timeit.timeit('reduction_(list(np.random.randint(-500,500, size=(65,65))))', globals=globals(), number=100))
print(size,timeit.timeit('reduction_(list(np.random.randint(-500,500, size=(70,70))))', globals=globals(), number=100))
print(size,timeit.timeit('reduction_(list(np.random.randint(-500,500, size=(75,75))))', globals=globals(), number=100))
print(size,timeit.timeit('reduction_(list(np.random.randint(-500,500, size=(80,80))))', globals=globals(), number=100))
print(size,timeit.timeit('reduction_(list(np.random.randint(-500,500, size=(85,85))))', globals=globals(), number=100))
print(size,timeit.timeit('reduction_(list(np.random.randint(-500,500, size=(90,90))))', globals=globals(), number=100))
print(size,timeit.timeit('reduction_(list(np.random.randint(-500,500, size=(95,95))))', globals=globals(), number=100))
print(size,timeit.timeit('reduction_(list(np.random.randint(-500,500, size=(100,100))))', globals=globals(), number=100))

48 99.68251711999983
48 124.79078937700069
48 153.87160797900015
48 190.1029419180004
48 226.5494457069999
48 269.77999819399975
48 307.8089243200002
48 368.6115573240004
48 435.73936380099985
48 477.45664181699976
48 499.2344346310001


## Perturbation Analysis

In [ ]:
print(timeit.timeit('reduction_(list(A+ np.round( np.random.normal(0, 1, size = (10, 10))).astype(int)))', globals=globals(), number=100))

2.0713615989999994


In [ ]:
print(timeit.timeit('reduction_(list(A+ np.round( np.random.normal(0, 5, size = (10, 10))).astype(int)))', globals=globals(), number=100))

2.0754793600000028


In [ ]:
print(timeit.timeit('reduction_(list(A+ np.round( np.random.normal(0, 10, size = (10, 10))).astype(int)))', globals=globals(), number=100))

2.15133818999999


In [ ]:
print(timeit.timeit('reduction_(list(A+ np.round( np.random.normal(0, 15, size = (10, 10))).astype(int)))', globals=globals(), number=100))

2.220706313999983


In [ ]:
print(timeit.timeit('reduction_(list(A+ np.round( np.random.normal(0, 20, size = (10, 10))).astype(int)))', globals=globals(), number=100))

2.2677699930000017


## 100 loop average

In [ ]:
variances = [0.005, 0.05, 0.1, 0.15]
for dim in range(10, 110, 10):
  print('n =',dim)
  A = np.random.randint(-500,500, size=(dim,dim))
  print('Without Perturbation',timeit.timeit('reduction_(list(A))', globals=globals(), number=1))
  for var in variances: 
    print('Perturbation with variance', var,':',timeit.timeit('reduction_(list(A+ np.round( np.random.normal(0, var*1000, size = (dim, dim))).astype(int)))', 
                                                              globals=globals(), number=1))

n = 10
Without Perturbation 0.028041857999937747
Perturbation with variance 0.005 : 0.022814168000081736
Perturbation with variance 0.05 : 0.014721179000048323
Perturbation with variance 0.1 : 0.019488371999955234
Perturbation with variance 0.15 : 0.024466083999982402
n = 20
Without Perturbation 0.10029082200003359
Perturbation with variance 0.005 : 0.11566721300005156
Perturbation with variance 0.05 : 0.15079269900002146
Perturbation with variance 0.1 : 0.13978861499992945
Perturbation with variance 0.15 : 0.081812760000048
n = 30
Without Perturbation 0.4132371800000101
Perturbation with variance 0.005 : 0.5549624600000698
Perturbation with variance 0.05 : 0.38297824300002503
Perturbation with variance 0.1 : 0.36606427399999575
Perturbation with variance 0.15 : 0.3010794660000329
n = 40
Without Perturbation 0.4308978249999882
Perturbation with variance 0.005 : 0.7619733989999986
Perturbation with variance 0.05 : 0.653329857000017
Perturbation with variance 0.1 : 0.7411982780000699
Per

## 5 loop Average

In [ ]:
variances = [0.005, 0.05, 0.1, 0.15]
for dim in range(10, 110, 10):
  print('n =',dim)
  A = np.random.randint(-500,500, size=(dim,dim))
  print('Without Perturbation',timeit.timeit('reduction_(list(A))', globals=globals(), number=5))
  for var in variances: 
    print('Perturbation with variance', var,':',timeit.timeit('reduction_(list(A+ np.round( np.random.normal(0, var*1000, size = (dim, dim))).astype(int)))', 
                                                              globals=globals(), number=5))

n = 10
Without Perturbation 0.07240303899970968
Perturbation with variance 0.005 : 0.08818638899992948
Perturbation with variance 0.05 : 0.07674716800011083
Perturbation with variance 0.1 : 0.11186524799995823
Perturbation with variance 0.15 : 0.11688001299989992
n = 20
Without Perturbation 0.43530981899994003
Perturbation with variance 0.005 : 0.4633359650001694
Perturbation with variance 0.05 : 0.5764894160001859
Perturbation with variance 0.1 : 0.6791513889997987
Perturbation with variance 0.15 : 0.8190752529999372
n = 30
Without Perturbation 1.2768563390000054
Perturbation with variance 0.005 : 1.4077524829999675
Perturbation with variance 0.05 : 1.5896210319997408
Perturbation with variance 0.1 : 1.2427238589998524
Perturbation with variance 0.15 : 1.4968038869997145
n = 40
Without Perturbation 2.641454993000025
Perturbation with variance 0.005 : 2.6851590360001865
Perturbation with variance 0.05 : 2.7836926769996353
Perturbation with variance 0.1 : 3.779873071000111
Perturbation 